In [9]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import pandas as pd
import plotly.express as px
import dash_bootstrap_components as dbc
import dizionari
import datetime

data = pd.read_csv("../data/ISTAT_Italian_Excursion_2014_2022.csv")

# CREO UN NUOVO DATAFRAME PIU' VERSATILE
# Nota: Nel nostro dataframe non sono presenti valori nulli [data.isnull().sum()]

# Rimozione di colonne ridondanti e superflui
colonne_da_eliminare = ["ID", "progind", "sesso", "staciv4", "condogg", "posiz4", "ateco3", "cond4", "ORARIO", "RAPP", "CODPROF7", "progesc", "PARTESC", "NPARTESC"]
df = data.drop(columns=colonne_da_eliminare)

# Rinomina delle colonne
df.rename(columns = dizionari.dizionario_nomi, inplace=True)

# Creazione di una colonna 'Data' nel formato (ANNO-mese)
df['Data'] = pd.to_datetime(df['Anno'].apply(str) + '-' + df['Mese'].apply(str), format='%Y-%m')
df["Data"] = df["Data"].dt.strftime("%Y-%m")

# Colonna Età nel formato (anno)
df['Età'] = df['Età'].map(dizionari.eta_mapping)


In [11]:
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.MATERIA])

radio_linechart = dbc.RadioItems(
    id="radio-linecharts-id",
    options=[
        {"label": "Età", "value": "Età"},
        {"label": "Estero", "value": "Destinazione (regioni italiane o paesi esteri)"},
    ],
    value="Età",
    inline=True
)

hist_feature_dropdown = dcc.Dropdown(
    id='hist-feature',
    options=[{'label': col, 'value': col} for col in df.columns],
    value='Mese'  # Valore predefinito
)

violin_feature_dropdown = dcc.Dropdown(
    id='violin-feature',
    options=[{'label': col, 'value': col} for col in df.columns],
    value='Mese'  # Valore predefinito
)

variabili_checklist = dbc.Checklist(
    id='correlation-checklist',
    options=[{'label': col, 'value': col} for col in df.columns],
    value=['Mese', 'Anno', 'Data'],  # Valori predefiniti
    inline=True
)
# Layout dell'app
app.layout = dbc.Container([
    html.H1("Analisi escursioni Italia"),
    dbc.Row([
        dbc.Col(
            dbc.Stack([
              html.Label("Seleziona la caratteristica dell'asse Y:"),
              radio_linechart,
              ])
            , width=6),
        dbc.Col(
            dbc.Stack([
              html.Label("Seleziona la caratteristica dell'asse Y:"),
              hist_feature_dropdown,
            ])
        , width=6, align="center"),
    ]),
    dbc.Row([
        dbc.Col(
            dbc.Stack([
              dcc.Graph(id='graph')
              ])
            , width=6),
        dbc.Col(
            dbc.Stack([
              dcc.Graph(id='histogram')
            ])
        , width=6),
    ]),
    dbc.Row([
        dbc.Col([
            dbc.Stack([
                html.Label("Select a feature to visualize with a violin plot:"),
                violin_feature_dropdown,
            ])
        ], width=4, align="center"),
        dbc.Col([
            dbc.Stack([
              html.Label("Seleziona le variabili per la heatmap di correlazione:"),
              variabili_checklist,
            ])
        ], width=8, align="center"),
    ]),
    dbc.Row([
        dbc.Col([
            dbc.Stack([
                dcc.Graph(id='violin-plot')
            ])
        ], width=4),
        dbc.Col([
            dbc.Stack([
              dcc.Graph(id='heatmap')
            ])
        ], width=8),
    ]),
])

# LINECHART - Evoluzione degli spostamenti negli anni (suddivisi per: [Età, Stati Esteri])
# Callback per il grafico a dispersione (scatter plot)
@app.callback(
    Output('graph', 'figure'),
    [Input('radio-linecharts-id', 'value')]
)
def update_line_chart(radio_linechart):
    conteggio = df.groupby(['Anno', radio_linechart]).size().reset_index(name='Numero escursioni')
    colori = ['#FF0000', '#FF4500', '#FF8C00', '#FFD700', '#ADFF2F', '#7FFF00', '#32CD32', '#008000']

    if radio_linechart == "Destinazione (regioni italiane o paesi esteri)":
        radio_linechart = "Stati Esteri"
        conteggio.rename(columns={"Destinazione (regioni italiane o paesi esteri)" : radio_linechart}, inplace=True)
        conteggio = conteggio[conteggio[radio_linechart] > 100]
        conteggio['Stati Esteri'] = conteggio[radio_linechart].map(dizionari.destinazione_regioni_stato_mapping)
        
        colori = [
            '#FF5733', '#33FF57', '#3357FF', '#FF33A6', '#A633FF', '#33FFA6',
            '#FF8C33', '#8CFF33', '#338CFF', '#FF338C', '#8C33FF', '#33FF8C',
            '#FF5733', '#33FF57', '#3357FF', '#FF33A6', '#A633FF', '#33FFA6',
            '#FF8C33', '#8CFF33', '#338CFF', '#FF338C', '#8C33FF', '#33FF8C',
            '#FF5733', '#33FF57', '#3357FF', '#FF33A6'
            ]

    print(conteggio)
    fig = px.line(conteggio,
                  x='Anno',
                  y='Numero escursioni',
                  color = radio_linechart,
                  title = f'Evoluzione degli spostamenti negli anni rispetto a [{radio_linechart}]',
                  color_discrete_sequence=colori)

    fig.update_layout(margin=dict(l=50, r=50, t=50, b=50))
    
    return fig

# Esecuzione dell'app
if __name__ == '__main__':
    app.run_server(debug=True, jupyter_mode="external")





Dash app running on http://127.0.0.1:8050/


    Anno    Età  Numero escursioni
0   2014   <=14                708
1   2014  15-24                356
2   2014  25-34                482
3   2014  35-44                855
4   2014  45-54                854
..   ...    ...                ...
67  2022  35-44                611
68  2022  45-54                787
69  2022  55-64                892
70  2022  65-74                605
71  2022   >=75                223

[72 rows x 3 columns]
     Anno    Stati Esteri  Numero escursioni
20   2014         Austria                 16
21   2014          Belgio                  1
22   2014         Francia                 19
23   2014        Germania                  2
24   2014        Slovenia                 14
..    ...             ...                ...
239  2022        Germania                  1
240  2022          Grecia                  1
241  2022        Slovenia                 12
242  2022         Croazia                  6
243  2022  Paesi Extra-UE                 20

[64 rows x 3 col